## IMPORT Library

In [766]:
import pandas as pd
import pymysql
import math
import numpy as np
import matplotlib.pyplot as plt

## Define functions

In [767]:
# 0,0,0,3,56,7,2,0,0,0,4,3,4,56,32,0,0,0,0 
# => [[3,56,7,2],[4,3,4,56,32]]
def get_list(queue):
    '''
     [0,0,0,3,56,7,2,0,0,0,4,3,4,56,32,0,0,0,0]
     => [[3,56,7,2],[4,3,4,56,32]]
    '''
    result = []
    tmp = []
    
    while(len(queue)>0):
        e = queue.pop(0)
        if e != 0 :
            tmp.append(e)
            # print(e)
        elif len(tmp)>0:
            result.append(tmp)
            tmp = []
            # print(result)
        else:
            continue
    
    return result

def get_index(queue):
    '''
     [0,0,0,3,56,7,2,0,0,0,4,3,4,56,32,0,0,0,0]
     => [[3,4,5,6],[10,11,12,13,14]]
    '''
    result = []
    tmp = []
    # print(len(queue))
    
    for i in range(len(queue)):
        e = queue.pop(0)
        if e != 0:
            tmp.append(i)
            # print(i)
        elif len(tmp)>0:
            result.append(tmp)
            tmp = []
            # print(result)
        else:
            continue
    # print("result: ", result)
    return result

def get_index_2(queue):
    '''
     [0,0,0,3,56,7,2,0,0,0,4,3,4,56,32,0,0,0,0]
     => [[3,4,5,6],[10,11,12,13,14]]
    '''
    result = []
    tmp = []
    # print(len(queue))
    
    
    for i in range(len(queue)):
        if len(queue) >=  4:
            e0 = queue.pop(0)
            e1 = queue[0]
            e2 = queue[1]
            # print(e0, e1, e2)

        if e0 != 0:
            tmp.append(i) 
            # print("01 if")
        elif len(tmp)>0 and (e1 != 0 or e2 != 0):
            tmp.append(i)
            # print("02 elif")
        elif len(tmp)>0 and e1 == 0 and e2 == 0:
            # print("03 elif")
            result.append(tmp)
            tmp = []
        else:
            # print("04 else")
            pass
#         print("tmp: ", tmp)
#         print("result", result)
#         print("   ")

    return result

    
    

## Read DB & get data

In [768]:
columns = ['DataSavedTime', 'Item']

In [769]:
conn = pymysql.connect(
    host='192.168.100.225',
    port=3306,
    user='root',
    password='its@1234',
    db='KIA_India_test',
    charset='utf8'
)

curs = conn.cursor()
# DL
# item = "Item055"
# exception = 10
# item = "Item016"
# exception = 20
# item = "Item018"
# exception = 20
# item = "Item032"
# exception = 15
# item = "Item033"
# exception = 20
# item = "Item034"
# exception = 10
# item = "Item035"
# exception = 10
# item = "Item044"
# exception = 10
# item = "Item045"
# exception = 10
item = "Item048"
exception = 20

# TL
# item = "Item001"
# exception = 30
# item = "Item002"
# exception = 20
# item = "Item013"
# exception = 10
# item = "Item014"
# exception = 15
# item = "Item021"
# exception = 20
# item = "Item022"
# exception = 20
# item = "Item023"
# exception = 20
# item = "Item024"
# exception = 20
# item = "Item025"
# exception = 20
# item = "Item026"
# exception = 10
# item = "Item027"
# exception = 20
# item = "Item028"
# exception = 20


sql = "select DataSavedTime, "+item+" from HisItemCurr WHERE DataSavedTime between \
    '2019-11-28 00:00:01' and '2019-11-28 12:00:00' ORDER BY DataSavedTime ASC"
curs.execute(sql)

rows=curs.fetchall()
# print(rows)
li = list(rows)
conn.close()
df = pd.DataFrame(li)
df.columns = columns
df = df.fillna(0)
print(df)

                DataSavedTime  Item
0     2019-11-28 09:30:00.000   0.0
1     2019-11-28 09:30:00.100   0.0
2     2019-11-28 09:30:00.200   0.0
3     2019-11-28 09:30:00.300   0.0
4     2019-11-28 09:30:00.400   0.0
...                       ...   ...
82940 2019-11-28 11:39:59.700   0.0
82941 2019-11-28 11:39:59.700   0.0
82942 2019-11-28 11:39:59.800   0.0
82943 2019-11-28 11:39:59.900   0.0
82944 2019-11-28 11:40:00.000   0.0

[82945 rows x 2 columns]


## Getting each wave form

In [770]:
Item = list(df['Item'])
Item_zero = [0 if v is None or np.isnan(v) else v for v in Item]
# print("sum(Item_zero) : ", sum(Item_zero))
result_i = get_index_2(Item_zero)
# print(result_i)
ln = []
# peak_list = []
sum_list = []
mean_list = []
start_list = []
stop_list = []

for i in range(len(result_i)):
    a = result_i[i]
    if len(a) <= exception:
        continue
    else:
        # print("Step: ", i, "len: ", len(a))
        # print(a)
        # print("Start Index:", a[0], "Stop Index:", a[-1])
        # print(len(a))

        stti = a[0]+exception
        # print(stti)
        stpi = a[-1]
        tmp_df = df.iloc[stti:stpi+1]
        # print(tmp_df.shape)
        # print(tmp_df)
        # print("tmp_df['Item']: ", tmp_df['Item'])
        sum_list.append(sum(tmp_df['Item']))
        # print("mean(avg)", np.mean(tmp_df['Item']))
        mean_list.append(np.mean(tmp_df['Item']))
        # print("peak", max(tmp_df['Item']))
        # peak_list.append(max(tmp_df['Item']))
        start = tmp_df['DataSavedTime'].iloc[0]
        start_list.append(start)
        stop = tmp_df['DataSavedTime'].iloc[-1]
        stop_list.append(stop)
        # print("Start:", tmp_df['DataSavedTime'].iloc[0], "Stop:", tmp_df['DataSavedTime'].iloc[-1])
        # print(stop - start)
        # plt.plot(tmp_df['DataSavedTime'], tmp_df['Item'])
        # plt.show()
        # ln.append(len(a))
    
# plt.hist(ln)
# print(len(ln))

dict_data = {'start': start_list, 'stop':stop_list, 'avg': mean_list, 'area': sum_list}

df_dict = pd.DataFrame(dict_data)
print(df_dict)

                     start                    stop        avg    area
0  2019-11-28 09:31:30.300 2019-11-28 09:31:37.000  12.032800  902.46
1  2019-11-28 09:31:58.300 2019-11-28 09:32:04.800  12.189014  865.42
2  2019-11-28 09:41:06.900 2019-11-28 09:41:13.600  13.313194  958.55
3  2019-11-28 09:41:35.300 2019-11-28 09:41:42.000  11.036316  838.76
4  2019-11-28 09:50:43.400 2019-11-28 09:50:50.300  11.985395  910.89
5  2019-11-28 09:51:11.900 2019-11-28 09:51:18.700  11.353733  851.53
6  2019-11-28 10:00:20.200 2019-11-28 10:00:26.800  12.952222  932.56
7  2019-11-28 10:00:48.700 2019-11-28 10:00:55.000  12.242206  832.47
8  2019-11-28 10:09:56.900 2019-11-28 10:10:03.600  12.050405  891.73
9  2019-11-28 10:10:25.200 2019-11-28 10:10:32.000  10.729189  793.96
10 2019-11-28 10:19:33.400 2019-11-28 10:19:40.000  13.030857  912.16
11 2019-11-28 10:20:02.100 2019-11-28 10:20:08.500  11.231304  774.96
12 2019-11-28 10:29:10.100 2019-11-28 10:29:17.000  12.641688  973.41
13 2019-11-28 10:29:

In [771]:
df_dict.to_csv(item + "_analysis_2019-11-28.csv", header=True)